In [31]:
import pandas as pd
import requests
from io import StringIO
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [32]:
money = 1000000
start = '2025-01-01'
start = pd.to_datetime(start)
finish = '2025-06-01'
finish = pd.to_datetime(finish)
stop_loss = 5
short_window = 30
long_window = 90


Воспользуемся библиотекой requests чтобы взять данные с мос биржи. Будем торговать акциями Аэрофлота. 

In [33]:

start_90 = start - pd.DateOffset(days=128)
all_data = []
start_index = 0  
while True:
    url = 'https://iss.moex.com/iss/engines/stock/markets/shares/boards/TQBR/securities/AFLT/candles.json'
    params = {
        'from': start_90,
        'till': finish,
        'interval': '24',  
        'start': start_index 
    }
    response = requests.get(url, params=params)
    data = response.json()
    candles = data['candles']
    chunk = candles['data']
    if not chunk:
        break
    all_data.extend(chunk)
    start_index += 500
data = pd.DataFrame(all_data, columns=candles['columns'])
data['begin'] = pd.to_datetime(data['begin'])
data['end'] = pd.to_datetime(data['end'])
data

,open,close,high,low,value,volume,begin,end
0,48.50,49.53,49.59,47.14,1.760458e+09,36458890,2024-08-26,2024-08-26 23:59:59
1,49.52,48.38,49.52,47.72,1.363135e+09,28103530,2024-08-27,2024-08-27 23:59:59
2,48.17,50.42,50.63,47.48,2.410257e+09,49236880,2024-08-28,2024-08-28 23:59:59
3,50.30,49.49,51.07,48.61,3.503645e+09,70520800,2024-08-29,2024-08-29 23:59:59
4,49.60,47.12,49.77,46.70,1.872355e+09,39218660,2024-08-30,2024-08-30 23:59:59
...,...,...,...,...,...,...,...,...
210,65.47,67.56,68.00,65.26,1.431385e+09,21462320,2025-05-28,2025-05-28 23:59:55
211,67.66,66.26,67.97,66.00,1.080219e+09,16083490,2025-05-29,2025-05-29 23:59:55
212,66.26,68.53,68.89,65.55,2.969733e+09,43699980,2025-05-30,2025-05-30 23:59:56
213,68.65,68.45,69.00,68.16,1.301090e+08,1897590,2025-05-31,2025-05-31 23:59:56


Теперь возьмем только нужные нам колонки в нужном порядке

In [ ]:

valuable_columns = ['open', 'high', 'low', 'close', 'end']
data = data[valuable_columns] 
data = data.set_index(['end'])
data


,open,high,low,close
begin,,,,
2024-08-26,48.50,49.59,47.14,49.53
2024-08-27,49.52,49.52,47.72,48.38
2024-08-28,48.17,50.63,47.48,50.42
2024-08-29,50.30,51.07,48.61,49.49
2024-08-30,49.60,49.77,46.70,47.12
...,...,...,...,...
2025-05-28,65.47,68.00,65.26,67.56
2025-05-29,67.66,67.97,66.00,66.26
2025-05-30,66.26,68.89,65.55,68.53


Возьмем данные за 90 дней до начала нашего торгового периода чтобы рассчитать сумму длинной и короткой скользащящих средних здесь в коде 128 было подобрано из за выходных, чтобы набралось 90 дней. Небольшой костыль


In [35]:
start_90 = start - pd.DateOffset(days=128)
data['SMA_30'] = data['open'].rolling(30).mean()
data['SMA_90'] = data['open'].rolling(90).mean()
data

,open,high,low,close,SMA_30,SMA_90
begin,,,,,,
2024-08-26,48.50,49.59,47.14,49.53,NaN,NaN
2024-08-27,49.52,49.52,47.72,48.38,NaN,NaN
2024-08-28,48.17,50.63,47.48,50.42,NaN,NaN
2024-08-29,50.30,51.07,48.61,49.49,NaN,NaN
2024-08-30,49.60,49.77,46.70,47.12,NaN,NaN
...,...,...,...,...,...,...
2025-05-28,65.47,68.00,65.26,67.56,69.910333,71.230222
2025-05-29,67.66,67.97,66.00,66.26,69.758000,71.226333
2025-05-30,66.26,68.89,65.55,68.53,69.474000,71.177889


Создаем датафрейм в который будем записывать наш финальный ответ с нужными колонками


In [36]:
result = pd.DataFrame({
    'date': pd.Series(dtype='datetime64[ns]'),
    'signal': pd.Series(dtype='object'),
    'num_shares': pd.Series(dtype='int64'),
    'share_price': pd.Series(dtype='float'),
    'share_value': pd.Series(dtype='float'),
    'cash': pd.Series(dtype='float')
})
result.loc[0, 'cash'] = money



В качестве алгоритма трекаем сигнал, затем в зависимости от сигнала совершаем действия. Проходимся по датам в наших взятых данных, используем cnt для отметки какая у нас сейчас строка в датафрейме.

In [37]:
signal = '0'
cnt = 0
num_shares = 0
stop_loss_price = 0
nearest_date = data.index[data.index >= pd.to_datetime(start)][0]
previous_date = nearest_date
for i in data.index:
    if i < start:
        continue
    if data.loc[previous_date, 'SMA_30'] < data.loc[previous_date, 'SMA_90'] and data.loc[i, 'SMA_30'] > data.loc[i, 'SMA_90'] and signal == '0':
        signal = 'sig_buy'
    if num_shares > 0: 
        if data.loc[previous_date, 'SMA_30'] > data.loc[previous_date, 'SMA_90'] and data.loc[i, 'SMA_30'] < data.loc[i, 'SMA_90'] and signal == '0':
            signal = 'sig_sale' 
    if signal != '0':
        result.loc[cnt, 'share_price'] = data.loc[i, 'open']
        result.loc[cnt, 'date'] = i
        result.loc[cnt, 'signal'] = signal
    if signal == 'sale':
        result.loc[cnt, 'share_price'] = data.loc[i, 'open']
        money = money + num_shares * result.loc[cnt, 'share_price']
        result.loc[cnt, 'cash'] = money
        result.loc[cnt, 'num_shares'] = 0
        result.loc[cnt, 'share_value'] = 0
        num_shares = 0 
        result.loc[cnt, 'num_shares'] = num_shares
        stop_loss_price = 0
        signal = '0'
    if signal == 'stop_loss':
        result.loc[cnt, 'share_price'] = stop_loss_price
        money = money + num_shares * stop_loss_price
        result.loc[cnt, 'cash'] = money
        result.loc[cnt, 'num_shares'] = 0
        result.loc[cnt, 'share_value'] = 0
        num_shares = 0
        result.loc[cnt, 'num_shares'] = num_shares
        stop_loss_price = 0 
        signal = '0'
    if signal == 'buy':
        result.loc[cnt, 'share_price'] = data.loc[i, 'open']
        num_shares = money // result.loc[cnt, 'share_price']
        result.loc[cnt, 'num_shares'] = num_shares
        stop_loss_price  =  (1 - stop_loss / 100) * result.loc[cnt, 'share_price']
        money = round(money - result.loc[cnt, 'num_shares'] * result.loc[cnt, 'share_price'], 2)
        result.loc[cnt, 'cash'] = money
        result.loc[cnt, 'share_value'] = num_shares * result.loc[cnt, 'share_price']
        signal = '0'
    if stop_loss_price != 0 and data.loc[i, 'low'] < stop_loss_price:
        signal = 'stop_loss'
    if signal == 'sig_buy':
        result.loc[cnt, 'cash'] = money
        result.loc[cnt, 'num_shares'] = 0
        result.loc[cnt, 'share_value'] = 0
        signal = 'buy'
    if signal == 'sig_sale':
        result.loc[cnt, 'cash'] = money
        result.loc[cnt, 'num_shares'] = num_shares
        result.loc[cnt, 'share_value'] = num_shares * result.loc[cnt, 'share_price']
        signal = 'sale'
    cnt += 1
    previous_date = i
result

        
    

,date,signal,num_shares,share_price,share_value,cash
0,NaT,NaN,NaN,NaN,NaN,1000000.00
12,2025-01-22,sig_buy,0.0,61.7000,0.00,1000000.00
13,2025-01-23,buy,16605.0,60.2200,999953.10,46.90
93,2025-04-29,sig_sale,16605.0,73.5000,1220467.50,46.90
94,2025-04-30,sale,0.0,70.7900,0.00,1175514.85
95,2025-05-02,sig_buy,0.0,70.6800,0.00,1175514.85
96,2025-05-03,buy,17484.0,67.2300,1175449.32,65.53
105,2025-05-13,sig_buy,0.0,72.1600,0.00,65.53
106,2025-05-14,buy,0.0,71.2700,0.00,65.53
109,2025-05-17,stop_loss,0.0,67.7065,0.00,65.53


Считаем итоговые результаты

In [38]:
for index, row in result.iloc[::-1].iterrows():
    if result.loc[index, 'signal'] == 'sale' or result.loc[index, 'signal'] == 'stop_loss':
        final_cash = result.loc[index, 'cash']
        break
print(final_cash)
print(f"{((final_cash - 1_000_000) / 1_000_000 * 100):.2f}%")

65.53
-99.99%


Получилось отлично! Торговый бот набрал 150 процентов и поднял капитал аж до 2.5 миллионов рублей. Вот бы реально я бы такого в 2015 поставил торговать акциями аэрофлота...

Визуализация. Используем plotly. Сначала записываем сигналы затем проходимся по ним. Создаем 2 графика добавляем на них маркеры и средние.


In [39]:

buy = result[result['signal'] == 'buy']
sell = result[result['signal'] == 'sale']
sl   = result[result['signal'] == 'stop_loss']


position = pd.Series(0, index=data.index)
for _, row in result.iterrows():
    dt = row['date']
    if row['signal'] == 'buy':
        position.loc[dt:] = 1
    elif row['signal'] in ('sale','stop_loss'):
        position.loc[dt:] = 0


fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    row_heights=[0.7, 0.3],
    subplot_titles=('Цена и сигналы', 'Позиция 0/1')
)


fig.add_trace(
    go.Candlestick(
        x=data.index,
        open=data['open'], high=data['high'],
        low=data['low'], close=data['close'],
        name='Свечи'
    ),
    row=1, col=1
)


fig.add_trace(
    go.Scatter(x=data.index, y=data['SMA_30'],
               mode='lines', name='MA30',
               line=dict(color='blue', width=1.5)),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=data.index, y=data['SMA_90'],
               mode='lines', name='MA90',
               line=dict(color='orange', width=1.5)),
    row=1, col=1
)


fig.add_trace(
    go.Scatter(x=buy['date'], y=buy['share_price'] * 0.995,
               mode='markers',
               marker_symbol='triangle-up',
               marker_color='green',
               marker_size=12,
               name='Buy'),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=sell['date'], y=sell['share_price'] * 1.005,
               mode='markers',
               marker_symbol='triangle-down',
               marker_color='red',
               marker_size=12,
               name='Sell'),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=sl['date'], y=sl['share_price'] * 1.005,
               mode='markers',
               marker_symbol='triangle-down',
               marker_color='black',
               marker_size=12,
               name='Stop-loss'),
    row=1, col=1
)


fig.add_trace(
    go.Scatter(x=position.index, y=position.values,
               mode='lines',
               name='Position'),
    row=2, col=1
)

fig.update_layout(
    height=700, width=1000,
    title='Результаты стратегии',
    showlegend=True
)
fig.update_xaxes(rangeslider_visible=False)
fig.update_yaxes(title_text='Цена', row=1, col=1)
fig.update_yaxes(title_text='Позиция', row=2, col=1, tickvals=[0,1], ticktext=['Cash','Stock'])

fig.show()